In [157]:
import numpy as np
import pandas as pd
import janitor
import os




In [158]:
ice = pd.read_excel('2025-ICLI-00019_2024-ICFO-39357_ERO Encounters_raw.xlsx', 
                    sheet_name=1, 
                    skiprows=6)

In [159]:
ice_clean = ice.clean_names()
ice_clean.head()

,event_date,responsible_aor,responsible_site,lead_event_type,lead_source,event_type,final_program,final_program_group,encounter_criminality,processing_disposition,...,final_order_date,birth_date,birth_year,citizenship_country,gender,event_landmark,alien_file_number,eid_case_id,eid_subject_id,unique_identifier
0,2024-08-03,HQ Area of Responsibility,ERO - Pacific Enforcement Response Center,Not applicable,NaN,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Prosecutorial Discretion,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Male,NaN,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
1,2024-10-18,Seattle Area of Responsibility,"PORTLAND, OR, DOCKET CONTROL OFFICE",Not applicable,NaN,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Not in Custody,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Unknown,NaN,"(b)(6), (b)(7)(c)","(b)(6), (b)(7)(c) (b)(7)(E)","(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
2,2024-11-21,Seattle Area of Responsibility,"PORTLAND, OR, DOCKET CONTROL OFFICE",Not applicable,NaN,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Warrant of Arrest/Notice to Appear,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Unknown,NaN,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
3,2025-02-16,HQ Area of Responsibility,ERO - Pacific Enforcement Response Center,Not applicable,NaN,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Not in Custody,...,NaT,"(b)(6), (b)(7)(c)",2002.0,MEXICO,Male,NaN,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",0000ad75abb300f8b89ada314a02a508fb886ad8
4,2025-03-20,New York City Area of Responsibility,"VARRICK, NY, SERVICE PROCESSING CENTER (DOCKET...",Not applicable,NaN,CAP Local,ERO Criminal Alien Program,ICE,2 Pending Criminal Charges,Detainer,...,NaT,"(b)(6), (b)(7)(c)",1980.0,VENEZUELA,Male,CAP - QUEENS CENTRAL BOOKING NY State,"(b)(6), (b)(7)(c)","(b)(6), (b)(7)(c) (b)(7)(E)","(b)(6), (b)(7)(C), (6)(7)(E)",0000b05116e18fc26472d546d43da50f24133e9c


### Per column, check missingness and unsual values

I replaced missing values (for character columns) with "Not applicable" or "Unknown", depending on what I thought made the most sense given my understanding of the variable. For columns in which the percentage of missing values was very low (i.e. < 1%), I dropped them. 


**Event Date**: Date of encounter

In [184]:

# unsual observation with event date at 2115-01-21
print(ice_clean.event_date.sort_values().tail(5))
print(ice_clean.event_date.sort_values().head(5))

ice_clean[ice_clean['event_date']=='2115-01-21']

# remove observation
ice_clean = ice_clean[ice_clean['event_date']!='2115-01-21']

700245   2025-06-27
299002   2025-06-27
83381    2025-06-27
381976   2025-06-27
700246   2025-06-27
Name: event_date, dtype: datetime64[ns]
477623   2024-08-01
198794   2024-08-01
616683   2024-08-01
486288   2024-08-01
354896   2024-08-01
Name: event_date, dtype: datetime64[ns]


**Responsible AOR**: Area of Responsibility (ICE field office)

In [185]:
print(ice_clean['responsible_aor'].unique())

# relatively small proportion of missing values of responsible AOR
print(ice_clean['responsible_aor'].isna().sum() / ice_clean.shape[0])

ice_clean = ice_clean.dropna(subset='responsible_aor')



['HQ Area of Responsibility' 'Seattle Area of Responsibility'
 'New York City Area of Responsibility'
 'Washington Area of Responsibility' 'Houston Area of Responsibility'
 'Newark Area of Responsibility' 'St. Paul Area of Responsibility'
 'Miami Area of Responsibility' 'San Francisco Area of Responsibility'
 'New Orleans Area of Responsibility' 'Dallas Area of Responsibility'
 'Denver Area of Responsibility' 'Buffalo Area of Responsibility'
 'Atlanta Area of Responsibility' 'Chicago Area of Responsibility'
 'San Diego Area of Responsibility' 'Boston Area of Responsibility'
 'El Paso Area of Responsibility' 'Harlingen Area of Responsibility'
 'Phoenix Area of Responsibility' 'Salt Lake City Area of Responsibility'
 'Philadelphia Area of Responsibility' 'Detroit Area of Responsibility'
 'San Antonio Area of Responsibility' 'Baltimore Area of Responsibility'
 'Los Angeles Area of Responsibility']
0.0


**Responsible Site**: Docket control office, which is a sub-office within an area of responsibility


In [186]:
print(ice_clean['responsible_site'].unique())

# no missing values within existing subset
print(ice_clean['responsible_site'].isna().sum() / ice_clean.shape[0])



['ERO - Pacific Enforcement Response Center'
 'PORTLAND, OR, DOCKET CONTROL OFFICE'
 'VARRICK, NY, SERVICE PROCESSING CENTER (DOCKET CONTROL OFFICE)'
 'ERO - Harrisonburg, VA Sub Office'
 'ERO - Montgomery County, TX Sub Office'
 'ICE ERO - MOUNT LAUREL NJ SUB-OFC' 'ST. PAUL, MN, DOCKET CONTROL OFFICE'
 'ERO - Bradenton, FL Sub Office'
 'SAN FRANCISCO, CA, DOCKET CONTROL OFFICE'
 'NEW ORLEANS, LA, DOCKET CONTROL OFFICE'
 'DALLAS, TX, DOCKET CONTROL OFFICE' 'ERO - Frederick, CO, Sub office'
 'ERO - Central Islip, NY Sub Office' 'BATAVIA CMD CTR NY DCO'
 'ERO - Stuart, FL Sub Office' 'ERO - Charlotte, NC Sub-Office'
 'CHICAGO, IL, DOCKET CONTROL OFFICE' 'TAMPA, FL, DOCKET CONTROL OFFICE'
 'ORLANDO, FL, DOCKET CONTROL OFFICE'
 'SAN DIEGO, CA, DOCKET CONTROL OFFICE' 'ERO - Manchester, NH Sub-Office'
 'ERO - Charleston, SC Sub-Office' 'ERO - Pecos, TX IHP Sub-Office'
 'HARLINGEN, TX, DOCKET CONTROL OFFICE' 'ERO - El Paso, TX SPC Sub-Office'
 "Jacksonville Sheriff's Office" 'ERO - Birmingham

**Lead event type**: unknown

In [187]:
# no missing values, and 'Not Applicable' seems to mean something
print(ice_clean['lead_event_type'].unique())
print(ice_clean[ice_clean['lead_event_type']=='Not applicable'].shape[0])

['Not applicable' 'CAP Federal' 'CAP Local' 'Alternatives to Detention'
 '287(g) Program Activities (Lead)' 'Non-Detained Docket Control'
 'Fugitive Operations (Lead)' 'CAP State' 'ERO Criminal Prosecutions'
 'Joint Criminal Alien Response Team' 'Detained Docket Control']
601617


**Lead source**: Unknown


In [188]:
print(ice_clean['lead_source'].unique())

print(ice_clean['lead_source'].isna().sum() / ice_clean.shape[0])

ice_clean['lead_source'] = ice_clean['lead_source'].fillna('Not applicable')

['Not applicable' 'BOP Referral (Bureau of Prisons)'
 'LEA Referral (Local Enforcement Agency)' 'Other Lead'
 'Priority Enforcement Program' 'Other Agency Referral'
 'National Security Referral' 'USMS Referral (US Marshals Service)'
 'Follow-up Investigation' 'DEA Referral (Drug Enforcement Agency)'
 'Anonymous Complaint' 'GAP(General Administrative Plan) special'
 'Deportations Referral' 'FBI Referral (Federal Bureau of Investigation)'
 'Foreign Fugitive Referral'
 'DOL(Department of Labor) Referral Using Form ESA-91' 'Link Analysis'
 'Visual Surveillance' 'CIS Criminal Referral to ERO' 'Joint Effort'
 'ATF Referral' 'Private Citizen' 'Written, Signed Complaint'
 'Inspections Referral' 'Customs Referral'
 'Foreign Local Enforcement Agency Referral'
 'DOI Referral (Department of the Interior)' 'Coast Guard Referral'
 'TECS Case Management number' 'National Targeting Plan' 'Informant'
 'DOL(Department of Labor) Referral Using Form Other Than ESA-91'
 'Non-Visual Surveillance' 'Dept of S

**Event Type**: roughly tracks the "Apprehension method" field in the arrests data, though many encounters do not result in arrests.

In [189]:
print(ice_clean['event_type'].unique())

# no actual NAs for current subset of data
print(ice_clean['event_type'].isna().sum() / ice_clean.shape[0])

['CAP Local' 'Alternatives to Detention' 'ERO Criminal Alien Program'
 'CAP Federal' 'Non-Detained Docket Control'
 'Administrative Non-Criminal Individual' 'Fugitive Operations (Event)'
 '287(g) Program Activities' 'Criminal Alien'
 'Administrative Criminal Alien' 'Fugitive Operations (Lead)' 'CAP State'
 'Assault' 'ERO Reprocessed' 'Probation and Parole (Event)'
 'ERO Criminal Prosecutions' 'Detained Docket Control'
 'Criminal Individual' 'OCDETF' 'U.S. Marshals' 'Not applicable'
 'Criminal Alien Smuggling' 'Administrative Alien Smuggling'
 'Worksite Investigation' 'Criminal Sanctions' 'Undocumented Alien'
 'Administrative Miscellaneous (Private Bill, Citation, etc)'
 'Criminal Organization' 'Unknown' 'Joint Criminal Alien Response Team'
 'Criminal Contraband Smuggling' 'Criminal Fraud' 'Human Trafficking'
 'GITMO Processing' 'ARO Waiver Submission' 'Administrative Fraud'
 'Administrative Seizure' 'Administrative Regulatory'
 'Administrative Sanctions' 'Mass Migration'
 '287(g) Progr

In [190]:
# 0.2% is 'Not applicable'
print(ice_clean[ice_clean['event_type']=='Not applicable'].shape[0]/ ice_clean.shape[0])

0.00021755714811204767


**Final Program**: The program associated with the arrest (not necessarily performed by ICE), which might also be described as the category of arrest. The values show the range of these categories, but key examples are the criminal alien program, which involves arrests in prisons and jails, the 287(g) program, which involves collaboration with a local agency, and border patrol.

In [191]:
print(ice_clean['final_program'].unique())

# no NAs
print(ice_clean['final_program'].isna().sum() / ice_clean.shape[0])

['ERO Criminal Alien Program' 'Fugitive Operations'
 'Non-Detained Docket Control' 'Alternatives to Detention'
 'Detention and Deportation' 'Detained Docket Control' '287G Program'
 'Mobile Criminal Alien Team' 'Juvenile' 'ERO Criminal Prosecutions'
 'Law Enforcement Area Response Units' '287g Task Force'
 'Violent Criminal Alien Section']
0.0


**Encounter Criminality**: This takes three values, corresponding to whether an individual has at least one criminal conviction, no criminal convictions but at least one criminal charge, or no charges or convictions ("other immigration violator")

In [192]:
print(ice_clean['encounter_criminality'].unique())
print(ice_clean['encounter_criminality'].isna().sum() / ice_clean.shape[0])

['3 Other Immigration Violator' '2 Pending Criminal Charges'
 '1 Convicted Criminal']
0.0


**Processing Disposition**: We are unsure how to understand the values in this field, especially in relation to case category.

Maybe decision that ICE made about an immigration case after the encounter?


In [193]:
print(ice_clean['processing_disposition'].unique())

# Low % of NAs
print(ice_clean['processing_disposition'].isna().sum() / ice_clean.shape[0])

ice_clean = ice_clean.dropna(subset='processing_disposition')

['Prosecutorial Discretion' 'Not in Custody'
 'Warrant of Arrest/Notice to Appear' 'Detainer' 'Other'
 'REINSTATEMENT OF DEPORT ORDER I-871' 'Expedited Removal (I-860)'
 'Not Amenable to Removal' 'Expedited Removal with Credible Fear'
 'Bag and Baggage' 'Foreign Born USC'
 'Reinstatement of Deportation Reasonable Fear' 'Voluntary Departure'
 'Voluntary Return' 'TURNED OVER TO'
 'Expedited Removal (I-860) - Full Scope' 'Paroled' 'VWP Removal'
 'ADMINISTRATIVE DEPORTATION I-851/I-851A' 'Notice to Appear (I-862)'
 'Crew Member (I-99) Removal' 'Stowaway'
 'Notice to Appear Detained (I-862)' 'Notice to Appear Released (I-862)'
 'I-210' 'Expedited Removal Temporary Final Rule'
 'Expedited Removal with Credible Fear - Full Scope' 'HSI Criminal Arrest'
 'Withdrawal (WD-Title 42)' 'VWP Removal (GUAM-CNMI)' 'Release']
0.0


**Case Status**: Includes some information about case type and status, most likely as of the date that the data was extracted.

In [194]:
print(ice_clean['case_status'].unique())

# 63% NAs. 
# TODO: ask if NAs mean pending status
print(ice_clean['case_status'].isna().sum() / ice_clean.shape[0])

ice_clean.loc[:, 'case_status'] = ice_clean['case_status'].fillna('Unknown')


['Unknown' 'ACTIVE' '8-Excluded/Removed - Inadmissibility'
 '6-Deported/Removed - Deportability' '3-Voluntary Departure Confirmed'
 'E-Charging Document Canceled by ICE' 'A-Proceedings Terminated'
 '9-VR Witnessed' '0-Withdrawal Permitted - I-275 Issued'
 'B-Relief Granted' '7-Died' '5-Title 50 Expulsion'
 'L-Legalization - Permanent Residence Granted'
 'Z-SAW - Permanent Residence Granted']
0.0


**Case Category**: This field includes combined information on case type and status. See https://journals.sagepub.com/doi/epdf/10.1177/233150241500300402 pp. 335-36 for descriptions of the values.

In [195]:
print(ice_clean['case_category'].unique())

# 63% NAs. 
# TODO: ask if NAs mean pending status
print(ice_clean['case_category'].isna().sum() / ice_clean.shape[0])

ice_clean.loc[:, 'case_category'] = ice_clean['case_category'].fillna('Unknown')

['Unknown' '[8A] Excludable / Inadmissible - Hearing Not Commenced'
 '[8B] Excludable / Inadmissible - Under Adjudication by IJ'
 '[16] Reinstated Final Order'
 '[8G] Expedited Removal - Credible Fear Referral'
 '[8C] Excludable / Inadmissible - Administrative Final Order Issued'
 '[2A] Deportable - Under Adjudication by IJ' '[8F] Expedited Removal'
 '[3] Deportable - Administratively Final Order'
 '[8K] Expedited Removal Terminated due to Credible Fear Finding / NTA Issued'
 '[5F] Unable to Obtain Travel Document' '[9] VR Under Safeguards'
 '[8E] Inadmissible - ICE Fugitive' '[5B] Removable - ICE Fugitive'
 '[5A] Referred for Investigation - No Show for Hearing - No Final Order'
 '[1A] Voluntary Departure - Un-Expired and Un-Extended Departure Period'
 '[8I] Inadmissible - ICE Fugitive - Expedited Removal'
 '[8D] Excludable / Inadmissible - Under Adjudication by BIA'
 '[11] Administrative Deportation / Removal'
 '[2B] Deportable - Under Adjudication by BIA'
 '[8H] Expedited Removal - 

**Departed Date**: Date of actual departure or deportation from the United States.

I'm assuming that missing values mean that the individual has not been deported. 

In [196]:
print(ice_clean['departed_date'].isna().sum() / ice_clean.shape[0])

0.8561618052663142


**Departure Country**: Country to which the individual was deported.

Since this variable depends on whether an individual was actually deported, I'm changing all the missing values to "Not applicable" and assuming the missing vallues mean that an individual has not yet been deported at the time of data collection

In [197]:
# % NAs for departure_country > % NAs for departed date
print(ice_clean['departure_country'].isna().sum()/ice_clean.shape[0])
ice_clean.loc[:, 'departure_country'] = ice_clean['departure_country'].fillna('Not applicable')

# looking for instances where an individual was deported but we don't know which country they went to
ice_clean[(ice_clean['departure_country'] == "Not applicable") & ~(ice_clean['departed_date'].isna())]

ice_clean.loc[
    (ice_clean['departure_country'] == "Not applicable") & ice_clean['departed_date'].notna(),
    'departure_country'
] = "Unknown"


0.0


**Final Order (Yes/No)** : This indicates whether an individual has a final order of removal, which means an order of removal that was either affirmed on appeal or not appealed, or that was issued without the involvement of an immigration judge (for example, in expedited removal). 

Replace NAs with "Unknown"

In [198]:
ice_clean.loc[:, 'final_order_yes_no'] = ice_clean['final_order_yes_no'].fillna('Unknown')

**Final Order Date**: Date of the final order of removal, which means an order of removal that was either affirmed on appeal or not appealed, or that was issued without the involvement of an immigration judge (for example, in expedited removal).

Since it is a date, I will keep the missing values as-is. 

In [199]:
print(ice_clean['final_order_date'].isna().sum() / ice_clean.shape[0])

0.8143407374328543


**Birth Year**

In [200]:
# low % of missing values
print(ice_clean['birth_year'].isna().sum() / ice_clean.shape[0])

ice_clean = ice_clean.dropna(subset='birth_year')

0.0


**Citizenship Country**

In [201]:

print(ice_clean['citizenship_country'].isna().sum() / ice_clean.shape[0])

0.0


**Gender**

In [202]:

print(ice_clean['gender'].isna().sum() / ice_clean.shape[0])

0.0


**Event Landmark**: Either an actual location or an ICE division associated with an arrest. See https://uwchr.github.io/ice-enforce/landmarks.html for details. Seems to be nested within AOR?

In [203]:
print(ice_clean['event_landmark'].isna().sum() / ice_clean.shape[0])
ice_clean.loc[:, 'event_landmark'] = ice_clean['event_landmark'].fillna('Unknown')

0.0


In [204]:
ice_clean.head()

,event_date,responsible_aor,responsible_site,lead_event_type,lead_source,event_type,final_program,final_program_group,encounter_criminality,processing_disposition,...,final_order_date,birth_date,birth_year,citizenship_country,gender,event_landmark,alien_file_number,eid_case_id,eid_subject_id,unique_identifier
0,2024-08-03,HQ Area of Responsibility,ERO - Pacific Enforcement Response Center,Not applicable,Not applicable,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Prosecutorial Discretion,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Male,Unknown,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
1,2024-10-18,Seattle Area of Responsibility,"PORTLAND, OR, DOCKET CONTROL OFFICE",Not applicable,Not applicable,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Not in Custody,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Unknown,Unknown,"(b)(6), (b)(7)(c)","(b)(6), (b)(7)(c) (b)(7)(E)","(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
2,2024-11-21,Seattle Area of Responsibility,"PORTLAND, OR, DOCKET CONTROL OFFICE",Not applicable,Not applicable,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Warrant of Arrest/Notice to Appear,...,NaT,"(b)(6), (b)(7)(c)",2006.0,HONDURAS,Unknown,Unknown,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",000080c6fa4a6ecd0b3beed3c559582c4205c50a
3,2025-02-16,HQ Area of Responsibility,ERO - Pacific Enforcement Response Center,Not applicable,Not applicable,CAP Local,ERO Criminal Alien Program,ICE,3 Other Immigration Violator,Not in Custody,...,NaT,"(b)(6), (b)(7)(c)",2002.0,MEXICO,Male,Unknown,"(b)(6), (b)(7)(c)",NaN,"(b)(6), (b)(7)(C), (6)(7)(E)",0000ad75abb300f8b89ada314a02a508fb886ad8
4,2025-03-20,New York City Area of Responsibility,"VARRICK, NY, SERVICE PROCESSING CENTER (DOCKET...",Not applicable,Not applicable,CAP Local,ERO Criminal Alien Program,ICE,2 Pending Criminal Charges,Detainer,...,NaT,"(b)(6), (b)(7)(c)",1980.0,VENEZUELA,Male,CAP - QUEENS CENTRAL BOOKING NY State,"(b)(6), (b)(7)(c)","(b)(6), (b)(7)(c) (b)(7)(E)","(b)(6), (b)(7)(C), (6)(7)(E)",0000b05116e18fc26472d546d43da50f24133e9c


In [205]:
ice_clean.to_csv('ice_encounters_cleaned.csv', index=False)